In [18]:
import pandas as pd
import numpy as np
df = pd.read_csv('car_ownership.csv')
df.head()

,monthly_salary,owns_car
0,22000,0
1,25000,0
2,47000,1
3,52000,0
4,46000,1


In [2]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

X = df[['monthly_salary']]
y = df['owns_car']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

model = LogisticRegression()
model.fit(X_train, y_train)
model.score(X_test, y_test)

0.8888888888888888

In [3]:
y_test.tolist()

[1, 0, 1, 0, 0, 0, 1, 1, 0]

In [4]:
model.predict(X_test)

array([1, 0, 1, 0, 0, 0, 0, 1, 0])

In [5]:
model.coef_, model.intercept_

(array([[0.00013621]]), array([-5.39725076]))

In [25]:
import math
def sigmoid(x):
    return 1 / (1 + math.exp(-x))

In [26]:
def prediction_function(salary, model):
    z = model.coef_*salary + model.intercept_
    y = sigmoid(z[0][0])
    return y

In [27]:
z = model.coef_*62000 + model.intercept_
z[0][0]

np.float64(3.047874614448302)

In [28]:
prediction_function(62000, model)

0.954690678813166